# Grupo 3 - Multi Layer Perceptron Modeling (Exercício 2)

---------------------------------------------

### Participants:
 - Francielle Vargas - 9527629
 - Lucas Nunes Sequeira - 9009642
 - Emanuel Huber - 12110113

#### Date: 26/08/2021

---------------------------------------------

#### Descrição e Instrução

Este notebook foi feito para a disciplina SCC5809 - Redes Neurais

No notebook contém

1. A implementação da classe **Perceptron**
2. A implementação da classe **PerceptronLayer**
3. A implementação da classe **MLP**
4. Resolução do problema XOR
5. Resolução do problema de Auto-Encoder

Para utilizá-lo, basta executar todas as células deste notebook.

_link de acesso ao colab: https://colab.research.google.com/drive/1W8kvzb0tthCJdDXfPy23U-oy7umToISG?usp=sharing_

### Libs

In [1]:
# Install Libs
!pip install numpy==1.19.5
!pip install tqdm==4.62.0
!pip install plotly==4.4.1
!pip install pandas==1.1.5

In [2]:
# Math lib
import numpy as np

# Log lib
from tqdm.auto import tqdm

# Visualization Lib
import plotly.express as px

# Copy for deepcopy
import copy

# Pandas for visualization
import pandas as pd

### Activation function

In [3]:
class Sigmoid(object):
  '''
  Sigmoid Activation Function
  
    f(x) =  1 / (1 + e^(-x))
  '''

  def __init__(self):

    # Last call data
    self.last_grad = 0
    self.last_input = 0
    self.last_output = 0

  def _update_last_call(self, x, y, grad = True):
    '''Update last call data'''

    # Gradient
    if grad:
      self.last_grad = self.gradient(x)

    # Update last input and output
    self.last_input = x
    self.last_output = y

  def __call__(self, x):
    '''Calculate sigmoid function of x'''

    if isinstance(x, list):
      x = np.array(x)

    return 1 / (1 + np.exp(-x))

  def calculate(self, x, grad: bool = True):
    '''Calculate sigmoid function of x'''

    # Calculation
    y = self(x)

    # Update last call data
    self._update_last_call(x, y, grad)

    return y

  def copy(self):
    return Sigmoid()

  def gradient(self, x):
    '''Calculate sigmoid gradient within x'''

    return self(x)*(1 - self(x))

In [4]:
class Swish(object):
  '''
  Swish Activation Function
  
    f(x) =  x * sigmoid(x)
  '''

  def __init__(self):

    self.sigmoid = Sigmoid()

    # Last call data
    self.last_grad = 0
    self.last_input = 0
    self.last_output = 0

  def _update_last_call(self, x, y, grad = True):
    '''Update last call data'''

    # Gradient
    if grad:
      self.last_grad = self.gradient(x)

    # Update last input and output
    self.last_input = x
    self.last_output = y

  def __call__(self, x):
    '''Calculate swish function of x'''

    if isinstance(x, list):
      x = np.array(x)

    return x*self.sigmoid(x)

  def calculate(self, x, grad: bool = True):
    '''Calculate swish function of x'''

    # Calculation
    y = self(x)

    # Update last call data
    self._update_last_call(x, y, grad)
    
    return y

  def copy(self):
    return Swish()

  def gradient(self, x):
    '''Calculate swish gradient within x'''

    return self.sigmoid(x)*(1 + x*(1 - self.sigmoid(x)))

In [5]:
class Relu(object):
  '''
  Relu Activation Function
  
    f(x) =  
      0 if x < 0
      x if x >= 0
  '''

  def __init__(self):

    # Last call data
    self.last_grad = 0
    self.last_input = 0
    self.last_output = 0

  def _update_last_call(self, x, y, grad = True):
    '''Update last call data'''

    # Gradient
    if grad:
      self.last_grad = self.gradient(x)

    # Update last input and output
    self.last_input = x
    self.last_output = y

  def __call__(self, x):
    '''Calculate relu function of x'''

    return np.where(x < 0, 0.0, x)

  def calculate(self, x, grad: bool = True):
    '''Calculate relu function of x'''

    # Calculation
    y = self(x)

    # Update last call data
    self._update_last_call(x, y, grad)
    
    return y

  def copy(self):
    return Relu()

  def gradient(self, x):
    '''Calculate relu gradient within x'''

    return np.where(x < 0, 0.0, 1)

### Loss Function

In [6]:
class MSE(object):
  '''
  Mean Squared Error Loss Function
  
    f(x) =  1/(2*m) * sum((ref_y - hyp_y)**2)
  '''

  def __init__(self):
    
    # Last call data
    self.last_grad = 0
    self.last_input = 0
    self.last_output = 0

  def _update_last_call(self, x, y, grad = True):
    '''Update last call data'''

    # Gradient
    if grad:
      self.last_grad = self.gradient(x[0], x[1])

    # Update last input and output
    self.last_input = x
    self.last_output = y

  def __call__(self, ref: np.ndarray, hyp: np.ndarray):
    '''Calculate mean squared error between ref and hyp'''

    if isinstance(ref, list):
      ref = np.array(ref)
    if isinstance(hyp, list):
      hyp = np.array(hyp)

    size = len(ref)

    # Quadratic Error Sum
    quadratic_sum = np.sum((ref - hyp)**2)

    return quadratic_sum/(2*size)

  def calculate(self, ref: np.ndarray, hyp: np.ndarray, grad: bool = True):
    '''Calculate mean squared error between ref and hyp'''

    # Calculation
    y = self(ref, hyp)

    # Update last call data
    self._update_last_call((ref, hyp), y, grad)

    return y

  def copy(self):
    return MSE()

  def gradient(self, ref: np.ndarray, hyp: np.ndarray):
    '''Calculate MSE gradient within hyp'''

    size = len(ref)

    return -np.sum(ref - hyp)/size

### Perceptron Class

In [7]:
class Perceptron():

  def __init__(self, input_size: int = 2, init_rule: str = 'zero', activation = None):
    '''Initializes Perceptron
    
      Params:
        input_size (int): size of input data
        init_rule (str): initialization parameter to set initial weight values
        activation (func): function to apply activation step
      '''

    # Perceptron size
    self.input_size = input_size
    self.size = input_size + 1

    # Initialize weights
    self.init_weights(init_rule)

    # Activation function
    if activation is None:
      activation = Sigmoid()
    self.activation = activation.copy()

    # Health check
    self._health_check()
  
  def _health_check(self):
    '''Perform a health check prediction'''

    # 1D (input_size)
    X = np.random.rand(self.input_size)
    self(X)

    # 2D (4 items, input_size)
    X = np.random.rand(4, self.input_size)
    self(X)

  def init_weights(self, init_rule: str = 'zero'):
    '''Initialize weights
    
      Params:
        init_rule (str): initialization parameter to set initial weight values
    '''

    # Assure initializarion rule
    assert init_rule in ['zero', 'rand'], "'init_rule' must be zero or rand"

    if init_rule == 'zero':
      # Zero values
      self.weights = np.zeros(self.size)
    
    elif init_rule == 'rand':
      # Random values in [-0.1, 0,1]
      self.weights = np.random.rand(self.size) - 0.5
      self.weights /= 0.5

  def set_weights(self, weights: np.ndarray):
    '''Update weights

      Params:
        weights (np.ndarray): array of weights
    '''

    # Assure perceptron size equals given weights size
    assert self.size == len(weights), f"Perceptron size ({self.size}) != weights size ({len(weights)})"

    # Update weights
    self.weights = weights.copy()

  def get_weights(self) -> np.ndarray:
    '''Return a copy of current weights'''

    # Return weights
    return self.weights.copy()

  def _add_bias_term(self, X: np.ndarray, value = 1.0, is_batch = True) -> np.ndarray:
    '''Add bias term to X values, ie:
       Given (x_i) in = [1, 0] -> out: [value, 1, 0]

      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
        value (float): Bias factor value. Default = 1.0
        is_batch (bool): Boolean to explicity that is or not a batch of items

      Returns:
        X (np.ndarray) with the bias term concatenated, eg:
    '''

    if is_batch:
      # Get batch_size
      batch_size = X.shape[0]

      # Add bias term
      X = np.concatenate([value*np.ones((batch_size, 1)), X], axis=1)
    
    else:
      # Add bias term
      X = np.concatenate([[value], X])

    return X

  def _prepare_input(self, X: np.ndarray) -> np.ndarray:
    '''Prepare input X
       
       1. Add batch dimension (if applies)
       2. Add bias term

      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
    '''

    # Make sure is a np.ndarray
    X = np.array(X)

    # Verify if it is sigle item and batch it
    if len(X.shape) == 1:
      # Add batch dimension
      X = np.expand_dims(X, 0)

    # Add bias term
    X = self._add_bias_term(X)

    return X


  def forward(self, X: np.ndarray) -> np.ndarray:
    '''Make a batch or single prediction
    
      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
        
      Returns:
        output (np.ndarray) logits
    '''
    
    # Prepare input data
    X = self._prepare_input(X)

    # Inner product of inputs and weigths (net)
    net = self.weights * X # multiplication
    net = np.sum(net, axis = 1) # sum reduction

    # Apply activation
    output = self.activation.calculate(net)

    return output

  def __call__(self, X: np.ndarray) -> np.ndarray:
    '''Make a batch or single prediction (runs forward method)
    
      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
        
      Returns:
        output (np.ndarray) logits
    '''

    return self.forward(X)

### Perceptron Layer Class

In [8]:
class PerceptronLayer():

  def __init__(self, input_size: int = 2, units: int = 2, init_rule: str = 'zero',
               activation = None):
    '''Initializes Perceptron Layer
    
      Params:
        input_size (int): size of input data
        units (int): perceptron units to use in layer
        init_rule (str): initialization parameter to set initial weight values
        activation (func): function to apply activation step
      '''

    # Perceptron Layer sizes
    self.input_size = input_size
    self.size = input_size + 1
    self.num_units = units
    self.output_size = self.num_units

    # Activation function
    if activation is None:
      activation = Sigmoid()
    self.activation = activation

    # Initialization rule
    self.init_rule = init_rule

    # Initialize units
    self._init_units()

    # Health check
    self._health_check()

  def _save_foward_transform(self, x, y):
    self.last_input = x
    self.last_output = y

  def _init_units(self):
    '''Initialize units of Layer'''

    # Layer list
    self.units = []

    # Iterate of amount of units
    for unit_id in range(self.num_units):
      self.units.append(
          Perceptron(
              input_size=self.input_size,
              init_rule=self.init_rule,
              activation=self.activation
          )
      )
  
  def _health_check(self):
    '''Perform a health check prediction'''

    # 1D (input_size)
    X = np.random.rand(self.input_size)
    self(X)

    # 2D (4 items, input_size)
    X = np.random.rand(4, self.input_size)
    self(X)


  def set_weights(self, weights: np.ndarray):
    '''Update weights per unit

      Params:
        weights (np.ndarray): array of weights; size: (num_units, len(unit.weights))
    '''

    # Assure weights have same length as number of units
    assert len(weights) == self.num_units, "Array of weigths must have shape (num_units, len(unit.weights))"

    for unit_weights, unit in zip(weights, self.units):

      # Update weights for each unit
      unit.set_weights(unit_weights)

  def get_weights(self) -> np.ndarray:
    '''Return a array of a copy of current weights per unit'''

    units_weights = []

    for unit in self.units:

      # Append unit weights
      units_weights.append(unit.get_weights())

    # Return weights
    return np.array(units_weights)

  def backward(self, delta, value):
    '''
    Apply backward propagation of the layer
    '''

    # Get gradient
    grad = self.activation.gradient(value)

    # Return new delta
    new_delta = (delta @ self.get_weights())[:, 1:] * grad

    return new_delta

  def forward(self, X: np.ndarray) -> np.ndarray:
    '''Make a batch or single prediction
    
      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
        
      Returns:
        output (np.ndarray) logits
    '''
    
    # Initalizate logits list (size of num units)
    logits = []

    for unit in self.units:
      
      # Apply foward in perceptron unit
      logit = unit(X)

      # Save logit
      logits.append(logit)

    # Return transpose logits
    logits = np.array(logits).T

    return logits

  def __call__(self, X: np.ndarray) -> np.ndarray:
    '''Make a batch or single prediction (runs forward method)
    
      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
        
      Returns:
        output (np.ndarray) logits
    '''

    y = self.forward(X)

    self._save_foward_transform(X, y)

    return y

### MultiLayerPerceptron Class

In [9]:
class MultiLayerPerceptron():

  def __init__(self, layers: list, loss_func = None):
    '''Initializes MultiLayerPerceptron Model
    
      Params:
        layers (list): list of perceptron layers
        loss_func (func): loss function to be applied
      '''

    # Perceptron Layers
    self.layers = layers
    self.num_layers = len(layers)

    # Input and output sizes
    self.input_size = layers[0].input_size
    self.output_size = layers[-1].output_size

    # Save loss
    if loss_func is None:
      self.loss_func = MSE()
    else:
      self.loss_func = loss_func

    # Health check
    self._health_check()
  
  def _health_check(self):
    '''Perform a health check prediction'''

    # Get first layer input size
    selfinput_size = self

    # 1D (input_size)
    X = np.random.rand(self.input_size)
    self(X)

    # 2D (4 items, input_size)
    X = np.random.rand(4, self.input_size)
    self(X)

  def _extend(self, vec):
    
    return np.hstack([np.ones((vec.shape[0], 1)), vec])

  def _backpropagate(self, x_input: np.ndarray, predicted_y: np.ndarray, 
                     reference_y: np.ndarray, learning_rate: float):
    '''
    Backpropagate loss to update each perceptron set of weigths in
    each layer of the model

    Params:
      x_input (np.ndarray): batch of input x
      reference_y (np.ndarray): batch of reference y's
      predicted_y (np.ndarray): batch of predicted y's
      learning_rate (float): learning rate param
    '''

    # Get first delta
    delta = predicted_y - reference_y

    # Get last predicted layer output
    last_output = predicted_y

    # Initialize weights dict variations
    dWs = {}

    # Iterate backwards over layers
    for i in range(-1, -len(self.layers), -1):
      
      # Get layer last output
      last_output = self.layers[i - 1].last_output

      # Update layer weights variation
      dWs[i] = delta.T @ self._extend(last_output)

      # Get new delta
      delta = self.layers[i].backward(delta, last_output)

    # Update layer weights variation (first layer)
    dWs[-self.num_layers] = delta.T @ self._extend(x_input)

    # Update each layer weights
    for k, dW in dWs.items():
      
      # Get current weights
      weights = self.layers[k].get_weights()
      weights -= learning_rate * dW

      # Update weights
      self.layers[k].set_weights(weights)

  def get_weights(self) -> np.ndarray:
    '''Return a list of arrays of a copy of current weights per layer and unit'''

    layer_weights = []

    for layer in self.layers:

      # Append unit weights
      layer_weights.append(layer.get_weights())

    # Return weights
    return layer_weights

  def _get_batch(self, X: np.ndarray, y: np.ndarray, batch_size: int):
    '''Generator of batch of items from X and y input data'''

    # X and y lengths must match
    assert len(X) == len(y), f"X (len = {len(X)}) and y (len = {len(y)}) lengths must match"

    # Produce batches
    batches = []

    # For each batch step append items
    for step in range(len(X)//batch_size + 2):
      
      # Get batch
      X_batch = X[step*batch_size:(step+1)*batch_size]
      y_batch = y[step*batch_size:(step+1)*batch_size]

      if len(X_batch) == 0: break

      batches.append({
          'X': X_batch,
          'y': y_batch
      })

    # Generate each batch pre-computed
    for batch in batches:

      # Return item
      yield batch

  def fit(self, X: np.ndarray, y: np.ndarray, learning_rate: float = 0.1,
          max_epochs: int = 5, stop_threshold: float = 1e-3, batch_size: int = 1):
    '''
    Fit the MLP model using a max_epochs steps, or when the stop_threshold
    is met

      Params:
        X (np.ndarray): a array of inputs, each input must match model input_size (number of features)
        y (np.ndarray): a array of target values (labels)
        learning_rate (float): hyperparameter to be used on backpropagation
        max_epochs (int): number of maximum epochs to iterate
        stop_threshold (float): number to be used to stop training if epoch loss is lower
        batch_size (int): size of each batch for the training steps

      Returns:
        history (dict): A dictionary containing training data over training as epoch loss
    '''

    # Assure X and y has same size
    assert len(X) == len(y), f"X (len = {len(X)}) and y (len = {len(y)}) lengths must match"
    assert len(y[0]) == self.layers[-1].num_units, f"y_i (len = {len(y[0])}) and output layer (len = {self.layers[-1].num_units}) lengths must match"

    # Epoch iterator
    iterator = tqdm(range(max_epochs))

    # Num items
    num_items = len(y)

    # History of train
    history = {'loss': []}

    for epoch in iterator:

      # Log epoch
      iterator.set_description(f'Epoch {epoch+1}/{max_epochs}')

      # Initialize epoch loss (mean absolute value)
      epoch_loss = 0
      
      # Iterate over all items
      for batch in self._get_batch(X, y, batch_size):

        # Retriever X and y batch
        input_x = batch['X']
        reference_y = batch['y']
        
        # Make prediction
        predicted_y = self(input_x)

        # Calculate error (loss)
        loss = self.loss_func.calculate(reference_y, predicted_y)

        # Add to epoch loss
        epoch_loss += batch_size * loss/num_items

        # Update weights
        # self._backpropagate(reference_y, predicted_y, learning_rate)
        self._backpropagate(input_x, predicted_y, reference_y, learning_rate)
      
      # Log epoch loss
      iterator.set_postfix({'loss': epoch_loss})

      # Append history
      history['loss'].append(epoch_loss)

      # Stop Threshold
      if epoch_loss < stop_threshold:
        print(f'>> Loss met stop condition (at epoch {epoch+1}): loss = {epoch_loss} < {stop_threshold}')
        break

    return history

  def forward(self, X: np.ndarray) -> np.ndarray:
    '''Make a batch or single prediction
    
      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
        
      Returns:
        output (np.ndarray) logits
    '''

    # Apply forward on each sequential layer
    for layer in self.layers:
      X = layer(X)
    
    return X

  def __call__(self, X: np.ndarray) -> np.ndarray:
    '''Make a batch or single prediction (runs forward method)
    
      Params:
        X (np.ndarray): Batch of items (2D array) or a item (1D array)
        
      Returns:
        output (np.ndarray) logits
    '''

    return self.forward(X)

### XOR Problem

#### Dataset

In [10]:
X = np.array([[1, 1], [0, 0], [0, 1], [1, 0]])
y = np.array([[0], [0], [1], [1]])

#### Build Model

In [11]:
# Layers
layer1 = PerceptronLayer(input_size=2, units=4, init_rule='rand', activation=Relu())
layer2 = PerceptronLayer(input_size=4, units=2, init_rule='rand', activation=Swish())
layer3 = PerceptronLayer(input_size=2, units=1, init_rule='rand', activation=Sigmoid())

layers = [layer1, layer2, layer3]

# Model
model = MultiLayerPerceptron(layers = layers, loss_func=MSE())

#### Train

In [12]:
# Model predictions before train:

print('>> Predictions before train:')
model(X)

>> Predictions before train:


array([[0.51623391],
       [0.37841334],
       [0.46270858],
       [0.47317163]])

In [13]:
hist = model.fit(X, y, max_epochs=2000, batch_size=1)

  0%|          | 0/2000 [00:00<?, ?it/s]

>> Loss met stop condition (at epoch 239): loss = 0.0009933750150525284 < 0.001


In [14]:
print('>> Predictions after train:')
model(X)

>> Predictions after train:


array([[0.03445139],
       [0.04707353],
       [0.95815299],
       [0.95366496]])

#### Training Visualization

In [15]:
fig = px.line(y=hist['loss'])
fig.update_layout(
    title = 'XOR | Loss vs Epochs',
    xaxis_title = 'Epoch',
    yaxis_title = 'Loss'
)

### Auto-Encoder Problem

#### Dataset

In [16]:
def identity_dataset(size: int = 9) -> tuple:
  '''
  Builds a dataset for auto-encoder training, in which
  the data (X <=> y) represents rows of identity matrix

  Ex: size = 3

    X = y = [
      [1, 0, 0],
      [0, 1, 0],
      [0, 0, 1]
    ]
  '''

  X = np.identity(size)
  y = np.identity(size)

  return X, y

In [17]:
def visualize_matrix(X):

  return pd.DataFrame(X)

In [18]:
X_8, y_8 = identity_dataset(8)
X_15, y_15 = identity_dataset(15)

#### Build Models

In [19]:
# Layers
layer1 = PerceptronLayer(input_size=8, units=3, init_rule='rand', activation=Swish())
layer2 = PerceptronLayer(input_size=3, units=8, init_rule='rand', activation=Relu())

layers = [layer1, layer2]

# Model
model_8 = MultiLayerPerceptron(layers = layers, loss_func=MSE())

In [20]:
# Layers
layer1 = PerceptronLayer(input_size=15, units=4, init_rule='rand', activation=Swish())
layer2 = PerceptronLayer(input_size=4, units=15, init_rule='rand', activation=Relu())

layers = [layer1, layer2]

# Model
model_15 = MultiLayerPerceptron(layers = layers, loss_func=MSE())

#### Train

##### Before Train

In [21]:
# Model predictions before train:

print('>> Predictions before train (Auto-Encoder 8)')
visualize_matrix(model_8(X_8))

>> Predictions before train (Auto-Encoder 8)


,0,1,2,3,4,5,6,7
0,0.644295,0.0,1.339091,0.479582,0.0,1.290386,1.874149,0.581963
1,0.408113,0.0,0.524639,0.061198,0.0,0.305845,0.942494,0.000000
2,0.000000,0.0,0.577586,0.000000,0.0,0.945446,1.012980,0.149977
3,0.697407,0.0,1.014143,0.369109,0.0,0.785715,1.620785,0.236968
4,1.691881,0.0,1.847840,1.374694,0.0,1.357167,0.946041,0.908644
5,0.000000,0.0,1.368284,0.109567,0.0,1.808394,2.224405,0.791059
6,1.404038,0.0,1.654793,1.185089,0.0,1.299082,0.551361,0.824311
7,0.715989,0.0,0.579684,0.309146,0.0,0.201187,0.406553,0.000000


In [22]:
print('>> Expected Outputs (Auto-Encoder 8)')
visualize_matrix(y_8)

>> Expected Outputs (Auto-Encoder 8)


,0,1,2,3,4,5,6,7
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
# Model predictions before train:

print('>> Predictions before train (Auto-Encoder 15)')
visualize_matrix(model_15(X_15))

>> Predictions before train (Auto-Encoder 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,0.218954,0.568630,0.000000,0.0,0.119015,0.000000,0.000000,0.0,0.006026,0.0,0.000000,1.402666,0.000000,0.627813
1,0.0,0.264905,0.386194,0.000000,0.0,0.083207,0.000000,0.000000,0.0,0.266482,0.0,0.000000,1.573474,0.000000,0.873773
2,0.0,0.000000,1.261920,0.341132,0.0,0.000000,0.120478,0.000000,0.0,0.000000,0.0,0.852118,0.963270,1.566816,0.015135
3,0.0,0.476437,0.152732,0.000000,0.0,0.102784,0.162329,0.000000,0.0,0.448002,0.0,0.000000,1.742998,0.000000,1.129389
4,0.0,0.216022,0.024076,0.000000,0.0,0.000000,0.303166,0.000000,0.0,0.183336,0.0,0.000000,1.675669,0.000000,1.173817
5,0.0,0.000000,0.956786,0.000000,0.0,0.000000,0.016430,0.000000,0.0,0.000000,0.0,0.065319,1.187055,0.756909,0.302496
6,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.567887,0.353632,0.0,0.169638,0.0,0.000000,1.690323,0.323872,1.304392
7,0.0,0.193185,0.000000,0.000000,0.0,0.000000,0.544689,0.058023,0.0,0.429362,0.0,0.000000,1.809264,0.000000,1.357500
8,0.0,0.034503,0.213680,0.000000,0.0,0.000000,0.290747,0.000000,0.0,0.201595,0.0,0.000000,1.597582,0.062665,1.011351
9,0.0,0.000000,0.390990,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,1.150988,0.185399,0.631619


In [24]:
print('>> Expected Outputs (Auto-Encoder 15)')
visualize_matrix(y_15)

>> Expected Outputs (Auto-Encoder 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


##### Training

In [25]:
hist_8 = model_8.fit(X_8, y_8, max_epochs=4000, batch_size=1, learning_rate=0.1)
hist_15 = model_15.fit(X_15, y_15, max_epochs=4000, batch_size=1, learning_rate=0.1)

  0%|          | 0/4000 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

>> Loss met stop condition (at epoch 143): loss = 0.0009641382923742789 < 0.001


##### After Train

In [26]:
# Model predictions after train:

print('>> Predictions after train (Auto-Encoder  8)')
visualize_matrix(model_8(X_8))

>> Predictions after train (Auto-Encoder  8)


,0,1,2,3,4,5,6,7
0,0.933008,0.075251,0.000000,0.439999,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.856953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,1.009392,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.572779,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.993762,0.000000,0.000340,0.000000
5,0.000000,0.000000,0.007442,0.000000,0.000000,1.147062,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000754,0.000000
7,0.000000,0.062842,0.000767,0.000000,0.000000,0.000000,0.000000,0.954543


In [27]:
# Model predictions after train:

print('>> Predictions after train (Auto-Encoder 15)')
visualize_matrix(model_15(X_15))

>> Predictions after train (Auto-Encoder 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.998483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.977634,0.000000,0.015886,0.000000,0.000000,0.000000,0.000000,0.002390,0.000000,0.014473,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.983509,0.000000,0.005273,0.011053,0.018424,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.983503,0.000000,0.013599,0.000000,0.012147,0.000000,0.000000,0.000000,0.003807,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.003579,0.000000,0.941165,0.000000,0.000000,0.000000,0.000000,0.013959,0.042126,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.006767,0.000000,0.977027,0.000000,0.000000,0.000000,0.000000,0.025459,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.991904,0.000000,0.000000,0.000000,0.000000,0.008614,0.000000,0.000000,0.013895
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.997098,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996705,0.000000,0.032539,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006526,0.957067,0.000000,0.000000,0.013843,0.000000,0.025940


#### Visualizing Train

In [28]:
fig = px.line(y=hist_8['loss'])
fig.update_layout(
    title = 'Auto Encoder - 8 | Loss vs Epochs',
    xaxis_title = 'Epoch',
    yaxis_title = 'Loss'
)

In [29]:
fig = px.line(y=hist_15['loss'])
fig.update_layout(
    title = 'Auto Encoder - 15 | Loss vs Epochs',
    xaxis_title = 'Epoch',
    yaxis_title = 'Loss'
)